In [33]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import h5py
filename = "C:/rppg/cohface_datatape_gray_32x8x8_train.h5"

with h5py.File(filename, "r") as f:
    print(len(f))
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    print("Keys: %s" % f.keys())
    # get first object name/key; may or may NOT be a group
    a_group_key = list(f.keys())[2]
    print(a_group_key)
    # get the object type for a_group_key: usually group or dataset
    print(type(f[a_group_key])) 

    # If a_group_key is a group name, 
    # this gets the object names in the group and returns as a list
    data = list(f[a_group_key])

    # If a_group_key is a dataset name, 
    # this gets the dataset values and returns as a list
    data = list(f[a_group_key])
    # preferred methods to get dataset values:
    ds_obj = f[a_group_key]      # returns as a h5py dataset object
    ds_arr = f[a_group_key][()]  # returns as a numpy array

5
Keys: <KeysViewHDF5 ['bvp', 'bvp_normalized', 'index', 'rr', 'vid']>
index
<class 'h5py._hl.dataset.Dataset'>


In [4]:
ds_arr.shape

(2709,)

In [36]:
train_tape = "C:/rppg/cohface_datatape_gray2_32x8x8_train.h5"
valid_tape = "C:/rppg/cohface_datatape_gray2_32x8x8_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)
train, valid = to_tf(train).cache(), to_tf(valid).cache()

In [28]:
# for RGB
model = keras.Sequential([
    layers.Input(shape=(32, 8, 8, 3)),
    layers.LayerNormalization(axis=(1,)),
    layers.Conv3D(4, (2, 2, 2), (1, 2, 2), padding='same', activation='tanh'),
    layers.LayerNormalization(axis=(1,)),
    layers.Conv3D(2, (2, 2, 2), (1, 2, 2), padding='same', activation='tanh'),
    layers.LayerNormalization(axis=(1,)),
    layers.AvgPool3D((1, 2, 2)),
    layers.Conv3D(1, 1, 1),
    layers.Flatten(),
], name='NoobHeart')
model.compile(optimizer='adam', loss='mae')
print(f'Flops per frame:{get_flops(model, input_sig=[tf.TensorSpec([1, 32, 8, 8, 3])])/32:.0f}')
model.summary()

Flops per frame:6062
Model: "NoobHeart"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_normalization_22 (La  (None, 32, 8, 8, 3)       64        
 yerNormalization)                                               
                                                                 
 conv3d_15 (Conv3D)          (None, 32, 4, 4, 4)       100       
                                                                 
 layer_normalization_23 (La  (None, 32, 4, 4, 4)       64        
 yerNormalization)                                               
                                                                 
 conv3d_16 (Conv3D)          (None, 32, 2, 2, 2)       66        
                                                                 
 layer_normalization_24 (La  (None, 32, 2, 2, 2)       64        
 yerNormalization)                                               
                                    

In [35]:
model.fit(train.shuffle(2000).batch(128), validation_data=valid.batch(32), epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/NoobHeart_Cohface.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
model.load_weights('../weights/NoobHeart_Cohface.h5')

Epoch 1/10
     23/Unknown - 0s 17ms/step - loss: 0.8654
Epoch 1: val_loss improved from inf to 0.84314, saving model to ../weights\NoobHeart_Cohface.h5
23/23 [==============================] - 1s 32ms/step - loss: 0.8654 - val_loss: 0.8431
Epoch 2/10
21/23 [==========================>...] - ETA: 0s - loss: 0.8355
Epoch 2: val_loss improved from 0.84314 to 0.83223, saving model to ../weights\NoobHeart_Cohface.h5
23/23 [==============================] - 1s 29ms/step - loss: 0.8346 - val_loss: 0.8322
Epoch 3/10
22/23 [===========================>..] - ETA: 0s - loss: 0.8269
Epoch 3: val_loss improved from 0.83223 to 0.82650, saving model to ../weights\NoobHeart_Cohface.h5
23/23 [==============================] - 1s 30ms/step - loss: 0.8271 - val_loss: 0.8265
Epoch 4/10
20/23 [=========================>....] - ETA: 0s - loss: 0.8214
Epoch 4: val_loss improved from 0.82650 to 0.82340, saving model to ../weights\NoobHeart_Cohface.h5
23/23 [==============================] - 1s 31ms/step - lo

In [38]:
# Gray Scale
model = keras.Sequential([
    layers.Input(shape=(32, 8, 8)),
    layers.LayerNormalization(axis=(1,)),
    layers.Conv2D(4, (2, 2), (1, 2), padding='same', activation='tanh'),
    layers.LayerNormalization(axis=(1,)),
    layers.Conv2D(2, (2, 2), (1, 2), padding='same', activation='tanh'),
    layers.LayerNormalization(axis=(1,)),
    layers.AvgPool2D((1, 2)),
    layers.Conv2D(1, 1),
    layers.Flatten(),
], name='NoobHeart')
model.compile(optimizer='adam', loss='mae')
print(f'Flops per frame:{get_flops(model, input_sig=[tf.TensorSpec([1, 32, 8, 8])])/32:.0f}')
model.summary()

Flops per frame:1945
Model: "NoobHeart"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_normalization_25 (La  (None, 32, 8, 8)          64        
 yerNormalization)                                               
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 4, 4)          132       
                                                                 
 layer_normalization_26 (La  (None, 32, 4, 4)          64        
 yerNormalization)                                               
                                                                 
 conv2d_8 (Conv2D)           (None, 32, 2, 2)          34        
                                                                 
 layer_normalization_27 (La  (None, 32, 2, 2)          64        
 yerNormalization)                                               
                                    

In [39]:
model.fit(train.shuffle(2000).batch(128), validation_data=valid.batch(32), epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/NoobHeart_Cohface.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
model.load_weights('../weights/NoobHeart_Cohface.h5')

Epoch 1/10
     21/Unknown - 2s 11ms/step - loss: 0.8865
Epoch 1: val_loss improved from inf to 0.86856, saving model to ../weights\NoobHeart_Cohface.h5
23/23 [==============================] - 2s 43ms/step - loss: 0.8855 - val_loss: 0.8686
Epoch 2/10
18/23 [======================>.......] - ETA: 0s - loss: 0.8730
Epoch 2: val_loss improved from 0.86856 to 0.85692, saving model to ../weights\NoobHeart_Cohface.h5
23/23 [==============================] - 0s 13ms/step - loss: 0.8700 - val_loss: 0.8569
Epoch 3/10
20/23 [=========================>....] - ETA: 0s - loss: 0.8582
Epoch 3: val_loss improved from 0.85692 to 0.84910, saving model to ../weights\NoobHeart_Cohface.h5
23/23 [==============================] - 0s 13ms/step - loss: 0.8583 - val_loss: 0.8491
Epoch 4/10
16/23 [===================>..........] - ETA: 0s - loss: 0.8508
Epoch 4: val_loss improved from 0.84910 to 0.84369, saving model to ../weights\NoobHeart_Cohface.h5
23/23 [==============================] - 0s 14ms/step - lo

In [30]:
model.load_weights('../weights/NoobHeart.h5')

In [40]:
# Gray Scale
eval_on_dataset(test_set_COHFACE, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_Cohface_gray.h5')
get_metrics('../results/NoobHeart_Cohface_gray.h5',)
#get_metrics_HRV('../results/NoobHeart_CCNU_CCNU.h5',)

100%|████████████████████████████████████████████████████████████████████████████████| 164/164 [00:04<00:00, 38.68it/s]


{'Sliding window': {'MAE': 9.705, 'RMSE': 19.39, 'R': 0.1395},
 'Whole video': {'MAE': 7.935, 'RMSE': 17.808, 'R': 0.197}}

In [34]:
# Normal Scale
eval_on_dataset(test_set_COHFACE, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_Cohface.h5')
get_metrics('../results/NoobHeart_Cohface.h5',)

  0%|                                                                                          | 0/164 [00:00<?, ?it/s]


ValueError: Exception encountered when calling layer 'NoobHeart' (type Sequential).

Input 0 of layer "conv3d_15" is incompatible with the layer: expected min_ndim=5, found ndim=4. Full shape received: (32, 32, 8, 8)

Call arguments received by layer 'NoobHeart' (type Sequential):
  • inputs=tf.Tensor(shape=(32, 32, 8, 8), dtype=float32)
  • training=None
  • mask=None

In [31]:
eval_on_dataset(test_set_COHFACE, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_Cohface.h5')
get_metrics('../results/NoobHeart_Cohface.h5',)

  0%|                                                                                          | 0/164 [00:00<?, ?it/s]


ValueError: Exception encountered when calling layer 'NoobHeart' (type Sequential).

Input 0 of layer "conv3d_15" is incompatible with the layer: expected min_ndim=5, found ndim=4. Full shape received: (32, 32, 8, 8)

Call arguments received by layer 'NoobHeart' (type Sequential):
  • inputs=tf.Tensor(shape=(32, 32, 8, 8), dtype=float32)
  • training=None
  • mask=None

In [9]:
eval_on_dataset(test_set_CCNU_rPPG, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_CCNU_rPPG.h5')
get_metrics('../results/NoobHeart_CCNU_CCNU_rPPG.h5',)
#get_metrics_HRV('../results/NoobHeart_CCNU_CCNU_rPPG.h5',)

100%|████████████████████████████████████████████████████████████████████████████████| 164/164 [00:05<00:00, 29.22it/s]


{'Sliding window': {'MAE': 23.386, 'RMSE': 27.999, 'R': -0.14362},
 'Whole video': {'MAE': 24.386, 'RMSE': 28.787, 'R': -0.21976}}

In [7]:
eval_on_dataset(test_set_PURE, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_PURE.h5')
get_metrics('../results/NoobHeart_CCNU_PURE.h5',)
#get_metrics_HRV('../results/NoobHeart_CCNU_PURE.h5',)

100%|██████████| 59/59 [00:02<00:00, 22.77it/s]


{'Sliding window': {'MAE': 1.121, 'RMSE': 6.905, 'R': 0.95399},
 'Whole video': {'MAE': 0.438, 'RMSE': 0.701, 'R': 0.99968}}

In [8]:
eval_on_dataset(test_set_UBFC_rPPG2, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_UBFC.h5')
get_metrics('../results/NoobHeart_CCNU_UBFC.h5',)
#get_metrics_HRV('../results/NoobHeart_CCNU_UBFC.h5',)

100%|██████████| 42/42 [00:01<00:00, 26.05it/s]


{'Sliding window': {'MAE': 1.762, 'RMSE': 4.327, 'R': 0.96993},
 'Whole video': {'MAE': 1.155, 'RMSE': 1.695, 'R': 0.99633}}

In [11]:
eval_on_dataset(test_set_MMPD, model, 32, (8, 8), step=0.5, batch=32, save='../results/NoobHeart_CCNU_MMPD.h5')
get_metrics('../results/NoobHeart_CCNU_MMPD.h5', motion='Stationary', skin_color='3', light=['LED-high', 'Incandescent', 'LED-low'])

100%|██████████| 660/660 [00:35<00:00, 18.64it/s]


{'Sliding window': {'MAE': 4.962, 'RMSE': 9.607, 'R': 0.52993},
 'Whole video': {'MAE': 2.776, 'RMSE': 6.311, 'R': 0.76314}}

In [10]:
eval_on_dataset(test_set_COHFACE, model, 32, (8, 8), step=0.5, batch=32, save='../results/NoobHeart_CCNU_CHOHFACE.h5')
get_metrics('../results/NoobHeart_CCNU_CHOHFACE.h5',)

100%|████████████████████████████████████████████████████████████████████████████████| 164/164 [00:10<00:00, 16.32it/s]


{'Sliding window': {'MAE': 24.009, 'RMSE': 28.604, 'R': -0.2251},
 'Whole video': {'MAE': 25.747, 'RMSE': 29.974, 'R': -0.31137}}